In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [15]:
import cv2
import torch
from yolort.utils.image_utils import letterbox, non_max_suppression
from yolort.utils import cv2_imshow, get_image_from_url, read_image_to_tensor,update_module_state_from_ultralytics


is_half = False
device = torch.device('cpu')
img_raw = get_image_from_url("https://gitee.com/zhiqwang/yolov5-rt-stack/raw/master/test/assets/bus.jpg")
ultralytics_weights_path = r"weights"
save_rt_weight_path = r'./weights/yolov5s_v40_updated.pt'

if os.path.exists(ultralytics_weights_path) is not True:
    os.makedirs(ultralytics_weights_path)
ultralytics_weights_path = f"{ultralytics_weights_path}/yolov5s.pt"
img = letterbox(img_raw, new_shape=(640, 640))[0]
img = read_image_to_tensor(img, is_half)
img = img.to(device)

#the weight's url in https://github.com/ultralytics/yolov5/releases
torch.hub.download_url_to_file(r'https://github.com/ultralytics/yolov5/releases/download/v4.0/yolov5s.pt',
                               ultralytics_weights_path,
                               progress=True)




In [ ]:
# config envrioment https://github.com/ultralytics/yolov5/issues/36
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [13]:
# 1.load ultiralytics model and inference
conf = 0.25
iou = 0.45

model = torch.hub.load('ultralytics/yolov5', 'custom', path=ultralytics_weights_path)
model.conf = conf  # confidence threshold (0-1)
model.iou = iou  # NMS IoU threshold (0-1)
model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
model.eval()
results = model(img.unsqueeze(dim=0))[0]
results = non_max_suppression(results, conf, iou, agnostic=True)[0]
print(results)



Using cache found in /home/featurize/.cache/torch/hub/ultralytics_yolov5_master
Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 2021-6-14 torch 1.8.1+cu111 CPU



tensor([[3.95028e+02, 2.28687e+02, 4.80437e+02, 5.25319e+02, 8.81427e-01, 0.00000e+00],
        [1.31801e+02, 2.40823e+02, 2.05202e+02, 5.10122e+02, 8.74423e-01, 0.00000e+00],
        [3.28450e+01, 2.39402e+02, 1.42193e+02, 5.31945e+02, 8.50409e-01, 0.00000e+00],
        [1.81174e+01, 1.36144e+02, 4.74266e+02, 4.48792e+02, 7.12929e-01, 5.00000e+00],
        [1.97863e-01, 2.94924e+02, 4.41639e+01, 5.27107e+02, 4.00531e-01, 0.00000e+00]])


In [16]:
# 2.convert yolov5 from ultralytics to rtstack
model = update_module_state_from_ultralytics(arch='yolov5s',
                                             version='v4.0',
                                             custom_path_or_model=ultralytics_weights_path,
                                             set_fp16=is_half)
torch.save(model.state_dict(),save_rt_weight_path)

Downloading: "https://github.com/ultralytics/yolov5/archive/v4.0.zip" to /home/featurize/.cache/torch/hub/v4.0.zip

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1

In [17]:
# 3.load rtstack model and inference
from yolort.models.yolo import yolov5_darknet_pan_s_r40 as yolov5s
model = yolov5s(pretrained=False, score_thresh=conf, nms_thresh=iou)
model.load_state_dict(torch.load(save_rt_weight_path))

# Load model
if is_half:
    model.half()  # to FP16
model.eval()
with torch.no_grad():
    model_out = model(img[None])

print(model_out)

[{'scores': tensor([0.88143, 0.87442, 0.85041, 0.71293, 0.40053]), 'labels': tensor([0, 0, 0, 5, 0]), 'boxes': tensor([[3.95028e+02, 2.28687e+02, 4.80437e+02, 5.25319e+02],
        [1.31801e+02, 2.40823e+02, 2.05202e+02, 5.10122e+02],
        [3.28450e+01, 2.39402e+02, 1.42193e+02, 5.31945e+02],
        [1.81174e+01, 1.36144e+02, 4.74266e+02, 4.48792e+02],
        [1.97876e-01, 2.94923e+02, 4.41640e+01, 5.27107e+02]])}]
